In [ ]:
# ybh-dlsk
CREATE OR REPLACE Function Ybh_f_khedkz_get_dlqk(p_101248_docid Number,
                                                 p_customid     Number,
                                                 p_returntype   Varchar2)
  Return Number Is

  v_lncqts Number := 0;
  v_lwcqts Number := 0;
  v_lncqje Number := 0;
  v_lwcqje Number := 0;
  v_ysje   Number := 0; --应收金额
  v_blsl   Number; --保量数量
  v_ljkdsl Number; --累计开单数量

Begin

  --获取带量收款及完成情况

  If p_101248_docid Is Null Or p_customid Is Null Or p_returntype Is Null Then
  
    Raise_application_error(-20000, '参数不能为空');
  
  End If;

  If p_returntype Not In ('量内超期天数',
                          '量内超期金额',
                          '量外超期天数',
                          '量外超期金额',
                          '报量完成情况',
                          '应收金额') Then
    Raise_application_error(-20000, 'p_returntype参数不对');
  End If;

  If p_returntype = '报量完成情况' Then
    --报量完成情况 0 报量内 1 报量外
  
    --获取报量数量
    Select nvl(d.blsl, 0)
      Into v_blsl
      From T_101248_DOC c, T_101248_Dtl d
     Where c.docid = d.docid(+)
       And c.docid = p_101248_docid
       And d.customid(+) = p_customid;
  
    --没有设置报量数量，都是量内
    If v_blsl = 0 Then
    
      Return 0;
    
    End If;
  
    -- 查找累计开单数量
    Select nvl(Sum(b.goodsqty), 0)
      Into v_ljkdsl
      From bms_sa_doc a, bms_sa_dtl b, T_101248_DOC c
    
     Where a.salesid = b.salesid
       And a.customid = p_customid
       And a.entryid = c.entryid
       And a.credate Between c.dlbegindate And c.dlenddate
       And b.goodsid = c.goodsid
       And a.usestatus = 1
       And c.docid = p_101248_docid;
    -- 累计开单数量大于报量数量 为报量外 累计开单数量小于报量数量 为报量内
    If v_ljkdsl > v_blsl Then
      Return 1;
    Else
      Return 0;
    End If;
  
  End If;

  Select nvl(Sum(a.BILLMONEY), 0) total_BILLMONEY, --总应收金额
         nvl(Sum(Case
                   When a.FLAG = '量外' And cqts > 0 Then
                    a.BILLMONEY
                   Else
                    0
                 End),
             0) lw_cq_BILLMONEY, --量外超期应收金额
         
         nvl(Max(Case
                   When a.FLAG = '量外' Then
                    a.cqts
                   Else
                    0
                 End),
             0) lw_cqts,
         nvl(Sum(Case
                   When a.FLAG = '量内' And cqts > 0 Then
                    a.BILLMONEY
                   Else
                    0
                 End),
             0) ln_cq_BILLMONEY, --量内超期应收金额
         nvl(Max(Case
                   When a.FLAG = '量内' Then
                    a.cqts
                   Else
                    0
                 End),
             0) ln_cqts
    Into v_ysje, v_lwcqje, v_lwcqts, v_lncqje, v_lncqts
  
    From v_lsqk_dl a
   Where a.docid = p_101248_docid
     And a.CUSTOMID = p_customid;

  If p_returntype = '量内超期天数' Then
    If v_lncqts > 0 Then
      Return v_lncqts;
    Else
      Return 0;
    End If;
  
  End If;

  If p_returntype = '量内超期金额' Then
    If v_lncqje > 0 Then
      Return v_lncqje;
    Else
      Return 0;
    End If;
  
  End If;

  If p_returntype = '量外超期天数' Then
    If v_lwcqts > 0 Then
      Return v_lwcqts;
    Else
      Return 0;
    End If;
  
  End If;

  If p_returntype = '量外超期金额' Then
    If v_lwcqje > 0 Then
      Return v_lwcqje;
    Else
      Return 0;
    End If;
  
  End If;

  If p_returntype = '应收金额' Then
    If v_ysje > 0 Then
      Return v_ysje;
    Else
      Return 0;
    End If;
  
  End If;

Exception

  When Others Then
  
    Return Null;
  
End;


In [ ]:
# ybh-khedkz-dl
CREATE OR REPLACE Function Ybh_f_khedkz_dl(p_101248_docid Number,
                                           p_salesid      Number,
                                           p_returntype   Varchar2)
  Return Varchar2 Is
  v_Rs       Varchar2(4000);
  v_customid Number;

  v_Rs_tmp Varchar2(4000);
  -- 带量欠款，按客户汇总
  Cursor cur_qk Is
    Select a.CUSTOMID,
           a.ENTRYID,
           
           a.XYHKTS, --协议回款天数
           a.CLHKTS, --超量回款天数
           a.DLMCPC, --带量名称批次
           Sum(a.BILLMONEY) total_BILLMONEY, --总应收金额
           Sum(Case
                 When a.FLAG = '量外' And cqts > 0 Then
                  a.BILLMONEY
                 Else
                  0
               End) lw_cq_BILLMONEY, --量外超期应收金额
           
           Max(Case
                 When a.FLAG = '量外' Then
                  a.cqts
                 Else
                  0
               End) lw_cqts,
           Sum(Case
                 When a.FLAG = '量内' And cqts > 0 Then
                  a.BILLMONEY
                 Else
                  0
               End) ln_cq_BILLMONEY, --量内超期应收金额
           Max(Case
                 When a.FLAG = '量内' Then
                  a.cqts
                 Else
                  0
               End) ln_cqts,
           Max(a.blje) blje --报量金额
    
      From v_lsqk_dl a
     Where a.docid = p_101248_docid
       And (a.CUSTOMID = v_customid Or nvl(v_customid, -1) = -1)
    
     Group By a.CUSTOMID, a.ENTRYID, a.XYHKTS, a.CLHKTS, a.DLMCPC;

  r_cur_qk cur_qk%Rowtype;

  v_kdje   Number := 0; --开单金额
  v_ysje   Number := 0; --应收金额+开单金额
  v_blje   Number := 0; --报量金额
  v_dlmcpc Varchar2(600);
  v_lncqts Number := 0;
  v_lwcqts Number := 0;

Begin

  v_Rs     := '';
  v_Rs_tmp := '';

  --获取带量设置对指定发货单的管控结果
  --p_salesid空代表跟销售单无关的数据
  --p_salesid为空时，p_returntype只能是开单管控提示信息，信誉情况

  If p_101248_docid Is Null Or p_returntype Is Null Then
  
    Raise_application_error(-20000,
                            'p_101248_docid或p_returntype参数不能为空');
  
  End If;

  If p_returntype Not In ('量内超期天数',
                          '量内超期金额',
                          '量外超期天数',
                          '量外超期金额',
                          '开单管控提示信息',
                          '信誉情况',
                          '应收金额') Then
    Raise_application_error(-20000, 'p_returntype参数不对');
  End If;

  --如果参数有销售单，只算该销售单涉及的客户，没有就算所有客户
  If p_salesid Is Not Null Then
  
    Select a.customid
      Into v_customid
      From bms_sa_doc a
     Where a.salesid = p_salesid;
  
    --只有销售单再有效时间内才算
    Select nvl(Sum(b.total_line), 0)
      Into v_kdje
      From bms_sa_doc a, bms_sa_dtl b, t_101248_doc c
     Where a.salesid = p_salesid
       And a.salesid = b.salesid
       And b.goodsid = c.goodsid
       And c.docid = p_101248_docid
       And a.credate Between c.dlbegindate And c.dlenddate;
  
  End If;

  Open cur_qk;
  Loop
    Fetch cur_qk
      Into r_cur_qk;
    Exit When cur_qk%Notfound;
    -- dbms_output.put_line(r_cur_qk.customid);
    --三种情况触发管控
    --量内超期
    --量外超期
    --应收金额超报量金额
  
    If r_cur_qk.ln_cqts > 0 Then
      v_lncqts := r_cur_qk.ln_cqts;
      v_Rs_tmp := '协议账期' || r_cur_qk.xyhkts || '量内超期' ||
                  r_cur_qk.ln_cqts || '量内超期金额' ||
                  r_cur_qk.ln_cq_BILLMONEY;
    End If;
  
    If r_cur_qk.lw_cqts > 0 Then
      v_lwcqts := r_cur_qk.lw_cqts;
      v_Rs_tmp := v_Rs_tmp || '超量账期' || r_cur_qk.clhkts || '量外超期' ||
                  r_cur_qk.lw_cqts || '量外超期金额' ||
                  r_cur_qk.lw_cq_BILLMONEY;
    End If;
  
    If r_cur_qk.blje > 0 And
       r_cur_qk.total_BILLMONEY + v_kdje > r_cur_qk.blje Then
      v_Rs_tmp := v_Rs_tmp || '应收金额大于报量金额';
    End If;
  
    --如果没有销售单则要计算所有客户，所以要展示客户id
    If v_customid Is Null And v_Rs_tmp Is Not Null Then
      v_Rs_tmp := '客户id' || r_cur_qk.customid || v_Rs_tmp;
    
    End If;
  
    If v_Rs_tmp Is Not Null Then
      v_Rs := v_rs || v_Rs_tmp;
    
    End If;
  
    v_Rs_tmp := '';
  
    v_ysje := r_cur_qk.total_BILLMONEY + v_kdje;
  
    If r_cur_qk.blje > 0 Then
      v_blje := r_cur_qk.blje;
    End If;
  
    v_dlmcpc := r_cur_qk.dlmcpc;
  
  End Loop;

  Close cur_qk;

  If v_Rs Is Not Null Then
    v_Rs := '带量批次:' || v_dlmcpc || v_Rs;
  End If;

  If p_returntype = '开单管控提示信息' Then
    If v_Rs Is Not Null Then
    
      Return v_rs;
    Else
      Return '';
    End If;
  
  End If;

  If p_returntype = '信誉情况' Then
    If v_Rs Is Not Null Then
    
      Return v_rs;
    Else
      Return '';
    End If;
  
  End If;

  If p_returntype = '量内超期天数' Then
    If v_lncqts > 0 Then
      Return v_lncqts;
    Else
      Return 0;
    End If;
  
  End If;

  If p_returntype = '量外超期天数' Then
    If v_lwcqts > 0 Then
      Return v_lwcqts;
    Else
      Return 0;
    End If;
  
  End If;

  If p_returntype = '应收金额' Then
    If v_ysje > 0 Then
      Return v_ysje;
    Else
      Return 0;
    End If;
  
  End If;
  -- Return v_rs;

Exception

  When Others Then
    --dbms_output.put_line(Sqlcode || Sqlerrm);
    Return '-1发生错误:' || dbms_utility.format_error_backtrace() || Sqlcode || Sqlerrm;
  
End;


In [ ]:
CREATE OR REPLACE Function Ybh_f_khedkz4(p_12102_seqid Number,
                                         p_salesid     Number,
                                         p_returntype  Varchar2) --返回类型 月均销售额 超期天数 超期金额 超信誉额度 信誉情况 开单信息...
 Return Varchar2 Is
  v_Rs       Varchar2(4000);
  v_customid Number;
  v_entryid  Number;
  v_count    Number;
  v_kcqed    Number; --可超期额度
  v_lscqqkje Number; --历史超期欠款金额
  v_bdcqqkje Number := 0; --本单超期欠款金额
  v_cqje     Number; --超期金额
  -- v_ljqked   Number; --累计欠款额度
  v_lszqkje Number; --历史总欠款额度
  v_bdqkje  Number := 0; --本单欠款金额
  v_cljqkje Number; --超累计欠款金额
  v_rjxse   Number; --日均销售额
  --v_qstzxs  Number; --缺省调整系数
  -- v_qsrjxse Number; --缺省日均销量
  -- v_ypqsrjxse Number; --药品缺省日均销量
  -- v_qxqsrjxse Number; --器械缺省日均销量

  --v_qtqsrjxse Number; --其他缺省日均销量
  v_xyts           Number; --信誉天数
  v_tzxs           Number; --调整系数
  v_xyedbz         Number; --信誉额度标准
  v_kzwd           Number; --控制维度
  v_nbfl           Number; --年报分类id
  v_nbflname       Varchar2(100); --年报分类
  v_cqts           Number; --超期天数
  v_yjxse          Number; --月均销售额
  v_tmp            Varchar(300);
  v_gklx           Number; --管控类型
  v_yjxszzl        Number; --月均销售增长率
  v_hkkdy          Number; --回款跨度月
  v_pdrq           Number; --判断日期
  v_zcybwskywrq    Date; --最长一笔未收款业务日期 add by ybh 2022/11/11
  v_tzxsyxq        Varchar2(50); --add by ybh 2023/4/26 浮动系数有效期
  v_zx_lsxyts      Number; --临时信誉天数 add by ybh 2023/6/1
  v_zx_invaliddate Varchar2(50); --临时信誉天数有效期 add by ybh 2023/6/1
  v_zx_lsxyts2     Number; --临时信誉天数 用于真实计算 add by ybh 2023/6/1
  v_source_tzxs    Number; --原始调整系数 --add by ybh 2023/7/5 用于展示在调整单信誉情况
Begin

  v_Rs := '';

  --获取管控信息对于指定销售发货单的管控结果
  --p_salesid空代表获取跟销售单无关的数据
  --20220812 可超额度去除，目前先保留，只是前台功能隐藏，所有值都是空的

  If p_12102_seqid Is Null Or p_returntype Is Null Then

    Raise_application_error(-20000,
                            'p_12102_seqid或p_returntype参数不能为空');

  End If;

  If p_returntype Not In ('超期天数',
                          '超期金额',
                          '超信誉额度',
                          '信誉情况',
                          '开单管控提示信息',
                          '应收账款余额',
                          '信誉天数',
                          '调整系数',
                          '信誉额度',
                          '月均销售额',
                          '信誉天数额度调整信誉情况',
                          '销售单管控生效类别',
                          '最长一笔未收款业务日期',
                          '真实应收账款余额') Then
    Raise_application_error(-20000, 'p_returntype参数不对');
  End If;

  --获取管控信息 管控类型 控制维度 用于判断是按客户还是按货品财务统计分类 天数还是自然月还是自然天数 可超期额度 信誉天数 调整系数
  Select t.flag,
         t.gklx,
         Case
           When t.kced2_validity > Sysdate And t.kced2 Is Not Null Then
            t.kced2
           Else
            0
         End kcqed,
         Case
           When t.gklx = 1 Then
            (t.hkkdy * 30 - 15 + t.pdrq)
           Else
            t.xyts
         End xyts, --信誉天数（自然月）：回款跨度月*30-15+判断日期
         --取有效期内的调整系数，没有值则取缺省调整系数 调整系数有效期为空代表调整系数小于缺省调整系数 如果都没有默认为1
         Case
           When t.tzxs Is Not Null And
                (t.tzxsyxq Is Null Or t.tzxsyxq > Sysdate) Then
            t.tzxs
           Else
            nvl(e.qstzxs, 1)
         End tzxs,
         t.nbfl,
         a.Ddlname,
         /* nvl(nvl(c.rjxse,
             decode(t.nbfl,
                    1,
                    e.ypqsrjxse,
                    2,
                    e.qxqsrjxse,
                    3,
                    e.qtqsrjxse,
                    e.qsrjxse)),
         0) rjxse, --日均销售额 没有销售额则取缺省日均销售额*/
         Case
           When c.rjxse Is Null Or c.rjxse <= 0 Then
            decode(t.nbfl,
                   1,
                   e.ypqsrjxse,
                   2,
                   e.qxqsrjxse,
                   3,
                   e.qtqsrjxse,
                   e.qsrjxse)
           Else
            c.rjxse
         End rjxse, --日均销售额  --add by ybh 2023/3/2 没有销售或者销售为0或负数 则取缺省销售额 避免信誉额度计算为负数或0

         nvl(c.yjxse, 0),
         c.yjxszzl,
         t.entryid,
         t.customid,
         t.hkkdy,
         t.pdrq,
         to_char(t.tzxsyxq, 'yyyy-mm-dd') tzxsyxq,
         nvl(t.zx_lsxyts, 0) zx_lsxyts,
         to_char(t.zx_invaliddate, 'yyyy-mm-dd') zx_invaliddate,
         Case
           When nvl(t.zx_invaliddate, Date '1900-01-01') > Sysdate And
                t.zx_lsxyts > 0 Then
            t.zx_lsxyts
           Else
            0
         End zx_lsxyts2, --只有有效期内的临时信誉天数才能用于计算
         t.tzxs
    Into v_kzwd,
         v_gklx,
         v_kcqed,
         v_xyts,
         v_tzxs,
         v_nbfl,
         v_nbflname,
         v_rjxse,
         v_yjxse,
         v_yjxszzl,
         v_entryid,
         v_customid,
         v_hkkdy,
         v_pdrq,
         v_tzxsyxq,
         v_zx_lsxyts,
         v_zx_invaliddate,
         v_zx_lsxyts2,
         v_source_tzxs
    From t_12102 t,
         (Select Ddlid, Ddlname From Pub_Ddl_Dtl Where Sysid = 100866) a,
         (Select f.entryid,
                 f.customid,
                 f.yjxse,
                 f.rjxse,
                 f.yjxszzl,

                 nvl(f.nbfl, -1) nbfl
            From YBH_T_KHXSE f) c, --每个月月初自动生成日均销售额 年报分类空代表整个客户销售额 外关联无法与nvl共用所有做嵌套处理

         (Select b.customid,
                 d.seqid,
                 d.xyts,
                 d.entryid,
                 d.qstzxs,
                 d.qsrjxse,
                 d.ypqsrjxse,
                 d.qxqsrjxse,
                 d.qtqsrjxse
            From t_101216 d, Pub_Customer b
           Where d.financeclass = b.financeclass) e --缺省信息
   Where t.seqid = p_12102_seqid
        --  And t.usestatus = 1
     And t.nbfl = a.Ddlid(+)
     And t.entryid = c.entryid(+)
     And t.customid = c.customid(+)
     And nvl(t.nbfl, -1) = c.nbfl(+)
     And t.entryid = e.entryid(+)
     And t.customid = e.customid(+);

  If p_returntype = '月均销售额' Then

    Return v_yjxse;

  End If;

  If p_returntype = '信誉天数' Then

    Return v_xyts;

  End If;

  If p_returntype = '调整系数' Then

    Return v_tzxs;

  End If;

  /* 信誉额度计算
  按自然月 （信誉天数（自然月）+15）*日均销售额*浮动系数
  按自然天数 信誉天数*日均销售额*浮动系数
  信誉天数（自然月）=回款跨度月*30-15+判断日期*/

  -- v_rjxse := nvl(v_rjxse, 0);
  -- v_tzxs  := nvl(v_tzxs, 0);

  If v_gklx = 1 Then
    v_xyedbz := (v_xyts + 15) * v_rjxse * v_tzxs;
  Else

    v_xyedbz := v_xyts * v_rjxse * v_tzxs;
  End If;

  If p_returntype = '信誉额度' Then

    Return v_xyedbz;

  End If;

  --按客户管控
  If v_kzwd = 1 Then

    --2.1 超期判断 历史超期欠款金额+本单超期欠款金额大于可超期额度 则判断为超期

    --获取历史超期欠款金额 超期判断算法分自然月 自然天数两种 设置只能二选一，不能两种共用 --modify by ybh 2022/11/2 超期日期增加判断金额大于0才计算
    Select nvl(Sum(c.Billmoney), 0),
           Max(Case
                 When c.BILLMONEY > 0 Then
                  Case
                    When t.gklx = 1 Then
                     Round(Sysdate -
                           (Trunc(Add_Months(c.Billdate, t.Hkkdy), 'mm') + t.Pdrq - 1)) -
                     v_zx_lsxyts2

                    Else
                     round(Sysdate - c.billdate - t.xyts - v_zx_lsxyts2)
                  End
                 Else
                  0
               End) cqts,
           Min(c.BILLDATE)
      Into v_lscqqkje, v_cqts, v_zcybwskywrq
      From v_lsqk c, t_12102 t
     Where c.Entryid = t.Entryid
       And c.Customid = t.Customid
       And ((Round(Sysdate -
                   (Trunc(Add_Months(c.Billdate, t.Hkkdy), 'mm') + t.Pdrq - 1)) -
           v_zx_lsxyts2 > 0

           And t.gklx = 1) Or
           (Sysdate - c.billdate > t.xyts + v_zx_lsxyts2 And t.gklx = 2))

          --  And c.sourcetable In ('BMS_SA_DOC', 'BMS_SA_SETTLE_DTL')
       And t.seqid = p_12102_seqid

    ;

    If p_salesid Is Not Null Then
      --获取本单超期欠款金额
      Select nvl(Sum(nvl(c.total_line, 0)), 0)
        Into v_bdcqqkje
        From bms_sa_dtl c, bms_sa_doc d, t_12102 t
       Where c.salesid = d.salesid
         And d.salesid = p_salesid
         And ((Round(Sysdate -
                     (Trunc(Add_Months(d.credate, t.Hkkdy), 'mm') + t.Pdrq - 1)) -
             v_zx_lsxyts2 > 0

             And t.gklx = 1) Or
             (Sysdate - d.credate > t.xyts + v_zx_lsxyts2 And t.gklx = 2))

         And t.seqid = p_12102_seqid

      ;
    End If;

    v_cqje := v_bdcqqkje + v_lscqqkje;
    --超期判断 历史超期欠款金额+本单超期欠款金额大于可超期额度 则判断为超期
    If v_cqje > 0 Then

      v_Rs := '该客户存在超期欠款，超期金额为：' || v_cqje || '超期天数为:' || v_cqts;

    End If;

    --2.2  超累计欠款额度判断 如果 历史总欠款金额+本单欠款金额大于累计欠款额度 则判断为超额度

    /*信誉额度标准计算逻辑
    按自然月（信誉天数（自然月）+15）*日均销售额*调整系数
    按自然天数 信誉天数*日均销售额*浮动系数
    日均销售额:
    控制维度为按客户时 该客户前3个自然月销售额/90 比如当前月为7月，则取4/5/6月销售额
    控制维度为安货品财务通及分类时 该客户该货品财务统计分类前3个自然月销售额/90
    如果没有销售额则取信誉天数标准功能设置该独立单元该客户缺省日均销量

    调整系数:  取在有效期（有效期为空代表长期）范围里面的调整系数，如果为空取该独立单元该客户的缺省调整系数

    信誉天数（自然月）：回款跨度月*30-15+判断日期
    */

    --日均销售额 没有销售取缺省日均销售额 月均销售额
    /* Select nvl(Sum(b.total_line) / 90, v_qsrjxse),
           nvl(Sum(b.total_line) / 3, 0) yjxse
      Into v_rjxse, v_yjxse
      From bms_sa_doc a, bms_sa_dtl b
     Where a.salesid = b.salesid
       And a.usestatus = 1
       And a.entryid = v_entryid
       And a.customid = v_customid
       And a.credate Between Trunc(add_months(Sysdate, -3), 'mm') And
           Trunc(Sysdate, 'mm')

    ;*/

    --获取历史总欠款金额
    Select nvl(Sum(c.Billmoney), 0)
      Into v_lszqkje
      From v_lsqk c
     Where /*c.sourcetable In ('BMS_SA_DOC', 'BMS_SA_SETTLE_DTL')
                                                                                                                                                   And*/
     c.entryid = v_entryid
     And c.customid = v_customid

    ;

    If p_salesid Is Not Null Then
      --获取本单欠款金额
      Select nvl(Sum(nvl(a.total_line, 0)), 0)
        Into v_bdqkje
        From bms_sa_dtl a
       Where a.salesid = p_salesid;

    End If;

    --超累计欠款金额 历史总欠款金额+本单欠款金额-信誉额度标准
    v_cljqkje := v_lszqkje + v_bdqkje - v_xyedbz;

    --如果历史总欠款金额+本单欠款金额大于累计欠款额度，则判断为超额度

    If v_cljqkje > 0 Then
      v_Rs := v_Rs || ' 该客户存在超信誉额度金额，金额为' || v_cljqkje;

    End If;

  Else

    --3 按年报分类管控
    If p_salesid Is Not Null Then
      --判断本单有无开管控的年报分类
      Select Count(1)
        Into v_count
        From bms_sa_dtl a, pub_goods b, bms_busiscope_def c
       Where a.salesid = p_salesid
         And a.goodsid = b.Goodsid
         And b.busiscope = c.scopedefid
         And c.lbfl = v_nbfl;
    End If;
    If v_count > 0 Or p_salesid Is Null Then

      --3.2 每个年报分类 进行超期超额判断

      --3.2.1 超期判断 历史超期欠款金额+本单超期金额大于可超期额度 则判断为超期 --modify by ybh 2022/11/2 超期日期增加判断金额大于0才计算

      --获取该年报分类历史超期欠款金额
      Select nvl(Sum(Billmoney), 0),
             nvl(Max(round(cqts)), 0),
             Min(BILLDATE)
        Into v_lscqqkje, v_cqts, v_zcybwskywrq
        From (Select Nvl(d.total_line, 0) Billmoney,
                     Case
                       When c.BILLMONEY > 0 Then
                        Case
                          When t.gklx = 1 Then
                           Round(Sysdate -
                                 (Trunc(Add_Months(c.Billdate, t.Hkkdy), 'mm') + t.Pdrq - 1)) -
                           v_zx_lsxyts2

                          Else
                           Sysdate - c.billdate - t.xyts - v_zx_lsxyts2
                        End
                       Else
                        0
                     End cqts,
                     c.BILLDATE
                From v_lsqk c, t_12102 t, bms_sa_dtl d, pub_goods_v e
               Where c.Entryid = t.Entryid
                 And c.Customid = t.Customid
                 And ((Round(Sysdate - (Trunc(Add_Months(c.Billdate, t.Hkkdy),
                                              'mm') + t.Pdrq - 1)) -
                     v_zx_lsxyts2 > 0

                     And t.gklx = 1) Or
                     (Sysdate - c.billdate > t.xyts + v_zx_lsxyts2 And
                     t.gklx = 2))
                 And c.sourcetable In ('BMS_SA_DOC')

                 And d.salesid = c.sourceid
                 And d.goodsid = e.Goodsid
                 And e.lbfl = v_nbfl
                 And t.seqid = p_12102_seqid

              Union All
              Select Nvl(c.billmoney, 0),
                     Case
                       When t.gklx = 1 Then
                        Round(Sysdate -
                              (Trunc(Add_Months(c.Billdate, t.Hkkdy), 'mm') + t.Pdrq - 1)) -
                        v_zx_lsxyts2

                       Else
                        Sysdate - c.billdate - t.xyts - v_zx_lsxyts2
                     End cqts,
                     c.BILLDATE
                From v_lsqk c, t_12102 t, BMS_SA_SETTLE_DTL f, pub_goods_v e
               Where c.Entryid = t.Entryid
                 And c.Customid = t.Customid
                 And ((Round(Sysdate - (Trunc(Add_Months(c.Billdate, t.Hkkdy),
                                              'mm') + t.Pdrq - 1)) -
                     v_zx_lsxyts2 > 0

                     And t.gklx = 1) Or
                     (Sysdate - c.billdate > t.xyts + v_zx_lsxyts2 And
                     t.gklx = 2))
                 And c.sourcetable In ('BMS_SA_SETTLE_DTL')
                 And f.sasettledtlid = c.sourceid
                 And f.goodsid = e.Goodsid
                 And e.lbfl = v_nbfl
                 And t.seqid = p_12102_seqid

              )

      ;
      --获取年报分类本单超期欠款金额
      If p_salesid Is Not Null Then
        Select nvl(Sum(c.total_line), 0)
          Into v_bdcqqkje
          From bms_sa_dtl c, bms_sa_doc d, t_12102 t, pub_goods_v e
         Where c.salesid = d.salesid
           And d.salesid = p_salesid
           And ((Round(Sysdate - (Trunc(Add_Months(d.credate, t.Hkkdy),
                                        'mm') + t.Pdrq - 1)) -
               v_zx_lsxyts2 > 0

               And t.gklx = 1) Or
               (Sysdate - d.credate > t.xyts + v_zx_lsxyts2 And t.gklx = 2))

           And c.goodsid = e.Goodsid
           And e.lbfl = v_nbfl
           And t.seqid = p_12102_seqid

        ;
      End If;
      v_cqje := v_bdcqqkje + v_lscqqkje;
      --超期判断 历史超期欠款金额+本单超期欠款金额大于可超期额度 则判断为超期
      If v_cqje > 0 Then

        v_Rs := ' 该客户年报分类:' || v_nbflname || '存在超期欠款金额，金额为' || v_cqje ||
                ' 超期天数为:' || v_cqts;

      End If;

      -- 3.2.2 超累计欠款额度判断 如果历史总欠款金额+本单欠款金额大于累计欠款额度 则判断为超额度
      --¸改用信誉额度判断 获取累计
      /*Select nvl(Max(nvl(t.kced, 0)), 0)
       Into v_ljqked
       From t_12102 t
      Where t.usestatus = 1
        And nvl(t.flag, 1) = 2
        And t.entryid = v_entryid
        And t.customid = v_customid
        And t.zx_finance_class = v_zx_finance_class;*/

      /*信誉额度标准计算逻辑
      按自然月（信誉天数（自然月）+15）*日均销售额*调整系数
    按自然天数 信誉天数*日均销售额*浮动系数
    日均销售额:
    控制维度为按客户时 该客户前3个自然月销售额/90 比如当前月为7月，则取4/5/6月销售额
    控制维度为安货品财务通及分类时 该客户该货品财务统计分类前3个自然月销售额/90
    如果没有销售额则取信誉天数标准功能设置该独立单元该客户缺省日均销量

    调整系数:  取在有效期（有效期为空代表长期）范围里面的调整系数，如果为空取该独立单元该客户的缺省调整系数

    信誉天数（自然月）：回款跨度月*30-15+判断日期
      */

      --年报分类日均销售额 没有销售取对应年报分类缺省日均销售额 月均销售额
      /* Select nvl(Sum(b.total_line) / 90,
                 decode(v_nbfl, 1, v_ypqsrjxse, 2, v_qxqsrjxse, v_qtqsrjxse)),
             nvl(Sum(b.total_line) / 3, 0) yjxse
        Into v_rjxse, v_yjxse
        From bms_sa_doc a, bms_sa_dtl b, pub_goods_v c
       Where a.salesid = b.salesid
         And b.goodsid = c.Goodsid
         And a.usestatus = 1
         And a.entryid = v_entryid
         And a.customid = v_customid
         And a.credate Between Trunc(add_months(Sysdate, -3), 'mm') And
             Trunc(Sysdate, 'mm')
         And c.lbfl = v_nbfl

      ;*/

      --获取历史总欠款金额
      Select nvl(Sum(Billmoney), 0)
        Into v_lszqkje
        From (Select Nvl(d.total_line, 0) Billmoney
                From v_lsqk c, bms_sa_dtl d, pub_goods_v e
               Where c.Entryid = v_entryid
                 And c.Customid = v_customid
                 And c.sourcetable In ('BMS_SA_DOC')
                 And d.salesid = c.sourceid
                 And d.goodsid = e.Goodsid
                 And e.lbfl = v_nbfl

              Union All
              Select Nvl(c.billmoney, 0)
                From v_lsqk c, BMS_SA_SETTLE_DTL f, pub_goods_v e
               Where c.Entryid = v_entryid
                 And c.Customid = v_customid
                 And c.sourcetable In ('BMS_SA_SETTLE_DTL')
                 And f.sasettledtlid = c.sourceid
                 And f.goodsid = e.Goodsid
                 And e.lbfl = v_nbfl

              )

      ;

      --获取本单欠款金额
      If p_salesid Is Not Null Then
        Select nvl(Sum(nvl(a.total_line, 0)), 0)
          Into v_bdqkje
          From bms_sa_dtl a, pub_goods_v e
         Where a.salesid = p_salesid
           And a.goodsid = e.Goodsid
           And e.lbfl = v_nbfl;
      End If;

      --超累计欠款金额 历史总欠款金额+本单欠款金额-累计欠款额度
      v_cljqkje := v_lszqkje + v_bdqkje - v_xyedbz;

      --如果历史总欠款金额 +本单欠款金额大于累计欠款额度，则判断为超额度

      If v_cljqkje > 0 Then
        v_Rs := v_Rs || ' 该客户年报分类:' || v_nbflname || '存在超信誉额度金额，金额为：' ||
                v_cljqkje;

      End If;

    End If;

  End If;

  --add by ybh 2022/11/11
  If p_returntype = '最长一笔未收款业务日期' Then
    Return to_char(v_zcybwskywrq, 'yyyy-mm-dd');

  End If;

  If p_returntype = '开单管控提示信息' Then

    If v_Rs Is Not Null Then

      v_rs := '根据12102功能单据id:' || p_12102_seqid || '的管控设置,' || v_rs;
    End If;

    Return v_rs;

  End If;

  If p_returntype = '超期天数' Then
    If v_cqje > 0 Then
      Return v_cqts;
    Else
      Return 0;
    End If;
  End If;

  If p_returntype = '超期金额' Then
    If v_cqje > 0 Then

      Return v_cqje;
    Else

      Return 0;
    End If;

  End If;
  If p_returntype = '超信誉额度' Then
    If v_cljqkje > 0 Then
      Return v_cljqkje;
    Else
      Return 0;
    End If;

  End If;

  --oa 超期、超额开单审批审批流 表单 信誉情况内容
  /* 格式 药品：月均销售额xx元；超期天数xx天；超期金额xx元
    器械：月均销售额xx元；超期天数xx天；超期金额xx元
   其他：月均销售额xx元；超期天数xx天；超期金额xx元
  上次申请时间：x月x日；上次申请日至今回款金额：xx元*/
  If p_returntype = '信誉情况' Then

    v_tmp := '信誉天数:' || v_xyts || '临时信誉天数:' || v_zx_lsxyts ||
             ' 临时信誉天数有效期:' || v_zx_invaliddate || ' 信誉额度:' || v_xyedbz ||
             ' 当前应收账款余额:' || nvl(v_lszqkje, 0);
    --只有超期 或 超额度才返回信誉情况
    If v_cljqkje > 0 Or v_cqje > 0 Then
      v_tmp := v_tmp || ' 月均销售额 ' || v_yjxse || ';';
      If v_cqje > 0 Then
        v_tmp := v_tmp || ' 超期天数 ' || round(v_cqts) || '天；超期金额  ' || v_cqje || ';';
      End If;
      If v_cljqkje > 0 Then
        v_tmp := v_tmp || ' 超信誉额度金额 ' || v_cljqkje || ';';
      End If;

      If v_nbflname Is Not Null Then
        v_tmp := v_nbflname || ':' || v_tmp;
      End If;

    End If;

    Return v_tmp;

  End If;

  --·返回销售单被管控的是什么类别 药品 器械 其他 -1代表客户
  If p_returntype = '销售单管控生效类别' Then

    If v_Rs Is Not Null Then

      Return nvl(v_nbfl, -1);
    End If;

  End If;

  If p_returntype = '应收账款余额' Then
    If v_lszqkje > 0 Then
      Return v_lszqkje;
    Else
      Return 0;
    End If;

  End If;

  --用于在功能 12102 展示负数的应收账款余额 --add by ybh 2023/3/2
  If p_returntype = 'ÕæÊµÓ¦ÊÕÕË¿îÓà¶î' Then

    Return v_lszqkje;

  End If;

  --oa审批流 信誉天数、额度调整申请（设置调整）表单字段信誉情况内容
  /*  月均销售额：（前三个月月均销售）：月均销售增长率：（前三个月月均增长率平均值）
  浮动系数：xx；信誉额度：xx元
  当前应收账款余额：xx元；超期应收账款：xx元
  回款跨度月：xx；回款判断日期：x日（该列仅在管控维度是自然月份时才显示）
  信誉天数：xx天；已超期天数：xx天 */

  If p_returntype = '信誉天数额度调整信誉情况' Then
    v_rs := '月均销售额：v_yjxse; 月均销售增长率：v_yjxszzl%;  浮动系数：v_tzxs; 浮动系数有效期：v_tzxsyxq  信誉额度：v_xyedbz 元;  当前应收账款余额：v_lszqkje 元; 超期应收账款：v_cqje 元;  v_replace_zry  信誉天数：v_xyts 天; 已超期天数v_cqts 天';
    v_rs := Replace(v_rs, 'v_tzxsyxq', v_tzxsyxq); --add by ybh 2023/4/26
    v_rs := Replace(v_rs, 'v_yjxse', v_yjxse);
    --用fm9999999990.99 解决0.几 前面0丢失问题
    v_rs := Replace(v_rs,
                    'v_yjxszzl',
                    to_char(round(v_yjxszzl, 4) * 100, 'fm9999999990.99'));
    v_rs := Replace(v_rs, 'v_tzxs', nvl(v_source_tzxs, v_tzxs)); --modify by ybh 2023/7/5 优先显示真实调整系数 避免只改浮动有效期时，不知道原来的浮动系数是多少
    v_rs := Replace(v_rs, 'v_xyedbz', v_xyedbz);
    v_rs := Replace(v_rs,
                    'v_lszqkje',
                    Case
                      When v_lszqkje > 0 Then
                       v_lszqkje
                      Else
                       0
                    End);

    v_rs := Replace(v_rs,
                    'v_cqje',
                    Case
                      When v_cqje > 0 Then
                       round(v_cqje, 2)
                      Else
                       0
                    End);
    v_rs := Replace(v_rs, 'v_xyts', v_xyts);
    v_rs := Replace(v_rs,
                    'v_cqts',
                    Case
                      When v_cqje > 0 Then
                       round(v_cqts)
                      Else
                       0
                    End);

    /*v_rs := '月均销售额' || v_yjxse || '月均销售增长率' || v_yjxszzl || chr(10) ||
    '浮动系数' || v_tzxs || '信誉额度' || v_xyedbz || '元' || chr(10) ||
    '当前应收账款余额' || Case
      When v_lszqkje > 0 Then
       v_lszqkje
      Else
       0
    End || '元：超期应收账款' || Case
      When v_cqje > 0 Then
       v_cqts
      Else
       0
    End || '元' || chr(10) || 'v_replace_zry信誉天数' || v_xyts ||
    '天：已超期天数' || Case
      When v_cqje > 0 Then
       v_cqts
      Else
       0
    End || '天';*/
    --管控类型是自然月时展示回款跨度月 判断日期
    If v_gklx = 1 Then

      v_rs := Replace(v_rs,
                      'v_replace_zry',
                      '回款跨度月' || v_hkkdy || '回款判断日期' || v_pdrq || '日');
    Else
      v_rs := Replace(v_rs, 'v_replace_zry', '');

    End If;
    v_rs := v_rs || ' 临时信誉天数:' || v_zx_lsxyts || ' 临时信誉天数有效期:' ||
            v_zx_invaliddate;

    Return v_rs;
  End If;

  /*Exception

  When Others Then
    --dbms_output.put_line(Sqlcode || Sqlerrm);
    Return '-1发生错误:' || dbms_utility.format_error_backtrace() || Sqlcode || Sqlerrm;*/

End Ybh_f_khedkz4;
